In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast

In [11]:
# parameters
customer_id = '134'
formatted_attribute = 'v_belt_outside_length_inches'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """V-Belt"""

regex_pattern = r"(?i)([^\.]\d* in\w* Outside Length)|(\d*\.\d* in\w* Outside Length)|(\d*\-\d*\/\d* in\w* Outside Length)|(\d*\.\d in\w* Outside Length)"


value='%n/a%'
customer_name='%motionindustriesus%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [12]:
metric = ' Inches'

range_string = """0 - 20 Inches
21 - 40 Inches
41 - 60 Inches
61 - 80 Inches
81 - 101 Inches
Greater than 100 Inches"""

In [13]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [14]:
df = query_from_file(file_name='./query/curated_products_VBOutDiamMM.sql', params=params)
df

attribute             value       customer_name  \
0      v_belt_outside_length_inches               n/a  motionindustriesus   
1      v_belt_outside_length_inches               n/a  motionindustriesus   
2      v_belt_outside_length_inches               n/a  motionindustriesus   
3      v_belt_outside_length_inches               n/a  motionindustriesus   
4      v_belt_outside_length_inches               n/a  motionindustriesus   
...                             ...               ...                 ...   
23221  v_belt_outside_length_inches  101 - 150 Inches  motionindustriesus   
23222  v_belt_outside_length_inches  101 - 150 Inches  motionindustriesus   
23223  v_belt_outside_length_inches  101 - 150 Inches  motionindustriesus   
23224  v_belt_outside_length_inches  101 - 150 Inches  motionindustriesus   
23225  v_belt_outside_length_inches  101 - 150 Inches  motionindustriesus   

      external_id  \
0        00450155   
1        00450156   
2        00450157   
3        00450158   
4        00450159   
...           ...   
23221    10544267   
23222    10544280   
23223    10544281   
23224    10544291   
23225    10544298   

                                                                                                                                                 long_desc  \
0      PolyFlex® 89132118 Polyflex Belt - 7M Size, 2 Sections, 46.2 in Outside Length, 0.2813 in Rib Width, 0.5625 in Top Width, 46.20 in Effective Length   
1      PolyFlex® 89132128 Polyflex Belt - 7M Size, 2 Sections, 50.1 in Outside Length, 0.2813 in Rib Width, 0.5625 in Top Width, 50.10 in Effective Length   
2                                                                                                                                 89132132 Polyflex Belt -   
3      PolyFlex® 89132140 Polyflex Belt - 7M Size, 2 Sections, 54.9 in Outside Length, 0.2813 in Rib Width, 0.5625 in Top Width, 54.90 in Effective Length   
4      PolyFlex® 89132150 Polyflex Belt - 7M Size, 2 Sections, 58.8 in Outside Length, 0.2813 in Rib Width, 0.5625 in Top Width, 58.80 in Effective Length   
...                                                                                                                                                    ...   
23221                                                                                                Hi-Power® V-Belt - B Section, 111.1 in Outside Length   
23222                                                                                                Hi-Power® V-Belt - B Section, 103.1 in Outside Length   
23223                                                                                                Hi-Power® V-Belt - B Section, 115.1 in Outside Length   
23224                                                                                             Tri-Power® V-Belt - CX Section, 102.62 in Outside Length   
23225                                                                                             Tri-Power® V-Belt - AX Section, 101.45 in Outside Length   

      resolution  
0       standard  
1       standard  
2       standard  
3       standard  
4       standard  
...          ...  
23221   standard  
23222   standard  
23223   standard  
23224   standard  
23225   standard  

[23226 rows x 6 columns]

In [15]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['0 - 25 Inches',
 '101 - 150 Inches',
 '151 - 200 Inches',
 '201 - 250 Inches',
 '251 - 300 Inches',
 '26 - 50 Inches',
 '301 - 400 Inches',
 '401 - 500 Inches',
 '501 - 600 Inches',
 '51 - 100 Inches',
 '601 - 700 Inches',
 'Greater than 700 Inches']

In [16]:
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

0        []
1        []
2        []
3        []
4        []
         ..
23221    []
23222    []
23223    []
23224    []
23225    []
Name: long_desc, Length: 23226, dtype: object

In [17]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [18]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
x=df[df['matches'].astype(str)!='[]']

In [23]:
x[x['value'].astype(str)!='n/a'][0:50]

attribute             value       customer_name  \
9698  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9699  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9700  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9701  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9702  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9703  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9704  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9705  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9706  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9707  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9708  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9709  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9710  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9711  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9712  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9713  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9714  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9715  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9716  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9717  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9718  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9719  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9720  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9721  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9722  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9723  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9724  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9725  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9726  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9727  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9728  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9729  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9730  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9731  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9732  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9733  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9734  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9735  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9736  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9737  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9738  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9739  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9740  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9741  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9742  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9743  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9744  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9745  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9746  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   
9747  v_belt_outside_length_inches  601 - 700 Inches  motionindustriesus   

     external_id  \
9698    00675829   
9699    00675830   
9700    00675835   
9701    00675836   
9702    00675868   
9703    00676044   
9704    

In [21]:
x['value'].explode().value_counts()

601 - 700 Inches    3941
101 - 150 Inches    2961
151 - 200 Inches    1588
301 - 400 Inches    1226
501 - 600 Inches    1096
401 - 500 Inches    1003
201 - 250 Inches     951
251 - 300 Inches     762
n/a                  689
Name: value, dtype: int64

In [7]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [8]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df

attribute value       customer_name external_id  \
0      polyflex_belt_effective_length   n/a  motionindustriesus    00450157   
1      polyflex_belt_effective_length   n/a  motionindustriesus    00450160   
2      polyflex_belt_effective_length   n/a  motionindustriesus    00456606   
3      polyflex_belt_effective_length   n/a  motionindustriesus    00466038   
4      polyflex_belt_effective_length   n/a  motionindustriesus    00466477   
...                               ...   ...                 ...         ...   
25998  polyflex_belt_effective_length   n/a  motionindustriesus    10544307   
25999  polyflex_belt_effective_length   n/a  motionindustriesus    10544308   
26000  polyflex_belt_effective_length   n/a  motionindustriesus    10544309   
26001  polyflex_belt_effective_length   n/a  motionindustriesus    10544310   
26002  polyflex_belt_effective_length   n/a  motionindustriesus    10544311   

                                               long_desc resolution  \
0                               89132132 Polyflex Belt -   standard   
1      PolyFlex® 89132155 Polyflex Belt - 7M Size, 2 ...   standard   
2      Micro-V® 917430114 Micro V-Belt - 14 Sections,...   standard   
3      Hi-Power® 90042152 V-Belt - C Section, 1 Band,...   standard   
4                                      93857125 V-Belt -   standard   
...                                                  ...        ...   
25998  Tri-Power® V-Belt - BX Section, 61.09 in Outsi...   standard   
25999  Tri-Power® V-Belt - BX Section, 71.22 in Outsi...   standard   
26000  Tri-Power® V-Belt - AX Section, 48.29 in Outsi...   standard   
26001  Tri-Power® V-Belt - AX Section, 28.32 in Outsi...   standard   
26002  Tri-Power® V-Belt - AX Section, 38.29 in Outsi...   standard   

                            matches  
0                                []  
1       [60.80 in Effective Length]  
2      [301.00 in Effective Length]  
3                                []  
4                                []  
...                             ...  
25998                            []  
25999                            []  
26000                            []  
26001                            []  
26002                            []  

[26003 rows x 7 columns]

In [9]:
df[df['matches'] != '[]']

attribute value       customer_name external_id  \
0      polyflex_belt_effective_length   n/a  motionindustriesus    00450157   
1      polyflex_belt_effective_length   n/a  motionindustriesus    00450160   
2      polyflex_belt_effective_length   n/a  motionindustriesus    00456606   
3      polyflex_belt_effective_length   n/a  motionindustriesus    00466038   
4      polyflex_belt_effective_length   n/a  motionindustriesus    00466477   
...                               ...   ...                 ...         ...   
25998  polyflex_belt_effective_length   n/a  motionindustriesus    10544307   
25999  polyflex_belt_effective_length   n/a  motionindustriesus    10544308   
26000  polyflex_belt_effective_length   n/a  motionindustriesus    10544309   
26001  polyflex_belt_effective_length   n/a  motionindustriesus    10544310   
26002  polyflex_belt_effective_length   n/a  motionindustriesus    10544311   

                                               long_desc resolution  \
0                               89132132 Polyflex Belt -   standard   
1      PolyFlex® 89132155 Polyflex Belt - 7M Size, 2 ...   standard   
2      Micro-V® 917430114 Micro V-Belt - 14 Sections,...   standard   
3      Hi-Power® 90042152 V-Belt - C Section, 1 Band,...   standard   
4                                      93857125 V-Belt -   standard   
...                                                  ...        ...   
25998  Tri-Power® V-Belt - BX Section, 61.09 in Outsi...   standard   
25999  Tri-Power® V-Belt - BX Section, 71.22 in Outsi...   standard   
26000  Tri-Power® V-Belt - AX Section, 48.29 in Outsi...   standard   
26001  Tri-Power® V-Belt - AX Section, 28.32 in Outsi...   standard   
26002  Tri-Power® V-Belt - AX Section, 38.29 in Outsi...   standard   

                            matches  
0                                []  
1       [60.80 in Effective Length]  
2      [301.00 in Effective Length]  
3                                []  
4                                []  
...                             ...  
25998                            []  
25999                            []  
26000                            []  
26001                            []  
26002                            []  

[26003 rows x 7 columns]

In [10]:
url='https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'

In [11]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [12]:
na_to_gsheet(df)

<ipython-input-11-c64bc6614460>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_matches[curation_col] = r'n/a'


succesfully saved to worksheet:`n/a` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


In [13]:
matches = df[df['matches'].astype(str) != '[]']

In [37]:

matches

attribute value       customer_name external_id  \
1      polyflex_belt_effective_length   n/a  motionindustriesus    00450160   
2      polyflex_belt_effective_length   n/a  motionindustriesus    00456606   
11     polyflex_belt_effective_length   n/a  motionindustriesus    00675759   
12     polyflex_belt_effective_length   n/a  motionindustriesus    00675769   
13     polyflex_belt_effective_length   n/a  motionindustriesus    00675775   
...                               ...   ...                 ...         ...   
24161  polyflex_belt_effective_length   n/a  motionindustriesus    08835961   
24162  polyflex_belt_effective_length   n/a  motionindustriesus    08835962   
24163  polyflex_belt_effective_length   n/a  motionindustriesus    08835963   
24164  polyflex_belt_effective_length   n/a  motionindustriesus    08835964   
24165  polyflex_belt_effective_length   n/a  motionindustriesus    08835971   

                                                                                                                                                 long_desc  \
1      PolyFlex® 89132155 Polyflex Belt - 7M Size, 2 Sections, 60.8 in Outside Length, 0.2813 in Rib Width, 0.5625 in Top Width, 60.80 in Effective Length   
2                              Micro-V® 917430114 Micro V-Belt - 14 Sections, M Size, 301.00 in Effective Length, 0.3750 in Rib Width, 5.2500 in Top Width   
11                                                   Poly V-Belt - 4 Sections, J Size, 89.00 in Effective Length, 0.0938 in Rib Width, 0.3750 in Top Width   
12                                                   Poly V-Belt - 6 Sections, J Size, 94.00 in Effective Length, 0.0938 in Rib Width, 0.5625 in Top Width   
13                                                  Poly V-Belt - 10 Sections, J Size, 84.00 in Effective Length, 0.0938 in Rib Width, 0.9375 in Top Width   
...                                                                                                                                                    ...   
24161                                                                                 Micro-V Belt - K Size, 89.33 in Effective Length, 0.807 in Top Width   
24162                                                                                 Micro-V Belt - K Size, 90.75 in Effective Length, 0.807 in Top Width   
24163                                                                                 Micro-V Belt - K Size, 85.32 in Effective Length, 0.807 in Top Width   
24164                                                                                 Micro-V Belt - K Size, 80.94 in Effective Length, 0.947 in Top Width   
24165                                                                                 Micro-V Belt - K Size, 40.75 in Effective Length, 0.947 in Top Width   

      resolution  matches Q:polyflex_belt_effective_length  
1       standard    60.80                              n/a  
2       standard   301.00          Greater than 100 Inches  
11      standard    89.00                  81 - 101 Inches  
12      standard    94.00                  81 - 101 Inches  
13      standard    84.00                  81 - 101 Inches  
...          ...      ...                              ...  
24161   standard    89.33                  81 - 101 Inches  
24162   standard    90.75                  81 - 101 Inches  
24163   standard    85.32                  81 - 101 Inches  
24164   standard    80.94                              n/a  
24165   standard    40.75                              n/a  

[1470 rows x 8 columns]

In [15]:
def remove_txt(lst):
    assert type(lst) == list, "make sure it's a list"
    tmp = []
    for measure in lst:
        val = re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',measure)
        if val != '':
            tmp.append(val.replace(",",''))
    if str(tmp) == '[]':
        return ''
    else:
        return '{:,}'.format(int(max(tmp)))

In [16]:
matches

attribute value       customer_name external_id  \
1      polyflex_belt_effective_length   n/a  motionindustriesus    00450160   
2      polyflex_belt_effective_length   n/a  motionindustriesus    00456606   
11     polyflex_belt_effective_length   n/a  motionindustriesus    00675759   
12     polyflex_belt_effective_length   n/a  motionindustriesus    00675769   
13     polyflex_belt_effective_length   n/a  motionindustriesus    00675775   
...                               ...   ...                 ...         ...   
24161  polyflex_belt_effective_length   n/a  motionindustriesus    08835961   
24162  polyflex_belt_effective_length   n/a  motionindustriesus    08835962   
24163  polyflex_belt_effective_length   n/a  motionindustriesus    08835963   
24164  polyflex_belt_effective_length   n/a  motionindustriesus    08835964   
24165  polyflex_belt_effective_length   n/a  motionindustriesus    08835971   

                                               long_desc resolution  \
1      PolyFlex® 89132155 Polyflex Belt - 7M Size, 2 ...   standard   
2      Micro-V® 917430114 Micro V-Belt - 14 Sections,...   standard   
11     Poly V-Belt - 4 Sections, J Size, 89.00 in Eff...   standard   
12     Poly V-Belt - 6 Sections, J Size, 94.00 in Eff...   standard   
13     Poly V-Belt - 10 Sections, J Size, 84.00 in Ef...   standard   
...                                                  ...        ...   
24161  Micro-V Belt - K Size, 89.33 in Effective Leng...   standard   
24162  Micro-V Belt - K Size, 90.75 in Effective Leng...   standard   
24163  Micro-V Belt - K Size, 85.32 in Effective Leng...   standard   
24164  Micro-V Belt - K Size, 80.94 in Effective Leng...   standard   
24165  Micro-V Belt - K Size, 40.75 in Effective Leng...   standard   

                            matches  
1       [60.80 in Effective Length]  
2      [301.00 in Effective Length]  
11      [89.00 in Effective Length]  
12      [94.00 in Effective Length]  
13      [84.00 in Effective Length]  
...                             ...  
24161   [89.33 in Effective Length]  
24162   [90.75 in Effective Length]  
24163   [85.32 in Effective Length]  
24164   [80.94 in Effective Length]  
24165   [40.75 in Effective Length]  

[1470 rows x 7 columns]

In [17]:
matches['matches'] = matches['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',str(measure)))

<ipython-input-17-6559beaf429e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['matches'] = matches['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',str(measure)))


In [18]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [19]:
matches['matches'] = matches['matches'].apply(lambda x: remove_fractions(x))

<ipython-input-19-53a3b7d5544b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['matches'] = matches['matches'].apply(lambda x: remove_fractions(x))


In [20]:
#matches['matches'].replace(r'\-','',regex=True,inplace=True)

In [21]:
matches['matches'].value_counts()

'115.00'    32
'22.00'     32
'50.00'     31
'28.00'     31
'26.00'     29
            ..
'33.5'       1
'39.78'      1
'33.78'      1
'26.89'      1
'45.28'      1
Name: matches, Length: 155, dtype: int64

In [22]:
#matches['matches'].replace(to_replace={"'32','44'":'44',"'55','44'":'55',"'50','35'":'50',"'45','65'":'65',"'32','630'":'630'},inplace=True)

In [23]:
matches['matches'] = matches['matches'].apply(lambda x: float(literal_eval(x)))

<ipython-input-23-5e96a50e59b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['matches'] = matches['matches'].apply(lambda x: float(literal_eval(x)))


In [24]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >= 0 and num <= 20:
        return "0 - 20 Inches"
        
    if num >= 21 and num <= 40:
        return "21 - 40 Inches"
        
    if num >= 41 and num <= 60:
        return "41 - 60 Inches"
        
    if num >= 61 and num <= 80:
        return "61 - 80 Inches"
        
    if num >= 81 and num <= 101:
        return "81 - 101 Inches"
        
    if num >= 100:
        return "Greater than 100 Inches"
        
    else:
        return r"n/a"



In [25]:
matches[curation_col] = matches['matches'].apply(range_app)

<ipython-input-25-7fac4195d173>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches[curation_col] = matches['matches'].apply(range_app)


In [26]:
matches.shape

(1470, 8)

In [27]:
gc = gcloud.Gsheets()

In [28]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

succesfully saved to worksheet:`ul` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


# send to the folder for upload

In [29]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)

In [30]:
looks_good('Motion Industries', attribute, df, matches)

<ipython-input-29-d72fa705f5c3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_matches[curation_col] = r'n/a'
